In [1]:
%env XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda/
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import numpy as np
import jax
from jax import random
import jax.numpy as jnp
import numpyro as npr
import matplotlib
import matplotlib.cm as cm
import matplotlib.dates as mdates
import tqdm
from math import *
import numpyro.distributions as dist
import seaborn as sns
sns.set_style('white')

matplotlib.rcParams['figure.figsize'] = (8,5)
matplotlib.rcParams['font.size'] = 10
matplotlib.rcParams['font.family'] = "serif"
matplotlib.rcParams['font.serif'] = 'Times'
matplotlib.rcParams['text.usetex'] = True
matplotlib.rcParams['lines.linewidth'] = 1
plt = matplotlib.pyplot

npr.set_platform('gpu')

env: XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/local/cuda/
env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


/home/agustinus/miniconda3/lib/python3.9/site-packages/jax/experimental/optimizers.py:28: FutureWarning: jax.experimental.optimizers is deprecated, import jax.example_libraries.optimizers instead
  warnings.warn('jax.experimental.optimizers is deprecated, '
/home/agustinus/miniconda3/lib/python3.9/site-packages/jax/experimental/stax.py:28: FutureWarning: jax.experimental.stax is deprecated, import jax.example_libraries.stax instead
  warnings.warn('jax.experimental.stax is deprecated, '


### Model

In [2]:
# Hyperparams
sigma = 0.09
nu = 12
NUM_STEPS = 100


def model(y, theta=None, rng_key=random.PRNGKey(1)):
    """ PyMC3 example http://num.pyro.ai/en/0.6.0/examples/stochastic_volatility.html """
    rng_key, rng_subkey = random.split(rng_key)

    num_steps = len(y) if y is not None else NUM_STEPS
    
    if theta is None:
        log_vol = npr.sample(
            'theta', dist.GaussianRandomWalk(scale=sigma, num_steps=num_steps), rng_key=rng_subkey
        )
    else:
        log_vol = theta
    
    rng_key, rng_subkey = random.split(rng_key)
    returns = npr.sample('y', dist.StudentT(df=nu, loc=0., scale=jnp.exp(log_vol)),
                         rng_key=rng_subkey, obs=y)
    
    if theta is None and y is None:  
        return log_vol  # Sample latent
    else:  
        return returns  # Given latent, sample y

#### VB

In [4]:
from numpyro.infer import SVI, Trace_ELBO
from numpyro.infer.autoguide import *


def train_vb_diag(rng_key, y, pbar=True):    
    guide = AutoDiagonalNormal(model)
    lr = 1e-3
    n_iter = 5000

    optimizer = npr.optim.ClippedAdam(step_size=lr)
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO(num_particles=100))
    svi_result = svi.run(rng_key, n_iter, y=y, progress_bar=pbar)
    
    return guide, svi_result.params


def train_vb_full(rng_key, y, pbar=True):    
    guide = AutoMultivariateNormal(model)
    lr = 5e-4  # Unstable with large lr
    n_iter = 10000  # Compensate with larger num. of iterations
    
    optimizer = npr.optim.ClippedAdam(step_size=lr)
    svi = SVI(model, guide, optimizer, loss=Trace_ELBO(num_particles=100))
    svi_result = svi.run(rng_key, n_iter, y=y, progress_bar=pbar)
    
    return guide, svi_result.params

In [5]:
POSTERIORS = ['vb_diag', 'vb_full', 'mcmc_long', 'mcmc_short']
POSTERIOR_FUNCS = {
    'vb_diag': train_vb_diag, 
    'vb_full': train_vb_full, 
}

### Baseline (Talts et al.)

In [6]:
M = 1e4
R = 31 
N = round(10.42*R)

print(f'M = {M}, R = {R}, N = {N}, N*R = {N*R}')


def run_baseline(posterior, rng_key=random.PRNGKey(1234)):    
    results = np.zeros([N, R+1, NUM_STEPS])
    
    for n in tqdm.trange(N):
        rng_key, *subkeys = random.split(rng_key, 6)
        
        theta_prior = dist.GaussianRandomWalk(scale=sigma, num_steps=NUM_STEPS).rsample(subkeys[0])
        results[n, 0, :] = theta_prior  # (NUM_STEPS,)
        
        y  = model(y=None, theta=theta_prior, rng_key=subkeys[1])
        
        # Get q(theta | y_t)
        if 'mcmc' not in posterior:
            guide, params = POSTERIOR_FUNCS[posterior](subkeys[2], y=y, pbar=False)
            theta = guide.sample_posterior(subkeys[3], params, (R,))['theta']
        else:
            num_warmup = 20 if 'long' in posterior else 5
            num_samples = 20 if 'long' in posterior else 5
            
            mcmc = MCMC(
                NUTS(model), 
                num_warmup=num_warmup, num_samples=num_samples, 
                progress_bar=False,
                num_chains=R,
                chain_method='vectorized'
            )
            mcmc.run(subkeys[4], y)
            
            # Last sample from each R chains
            theta = np.array(
                mcmc.get_samples(group_by_chain=True)['theta'][:, -1, :]
            ).squeeze()
            
            assert theta.shape == (R, NUM_STEPS)
            
        results[n, 1:, :] = theta  # (R, NUM_STEPS)
                
    return results

    
for posterior in POSTERIORS:    
    results = run_baseline(posterior=posterior)
    np.save(f'../../results/baseline/baseline_{posterior}.npy', results)

M = 10000.0, R = 31, N = 323, N*R = 10013


100%|██████████████████████████████████████████████████████████████████████| 323/323 [34:02<00:00,  6.32s/it]
